In [263]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Multiply, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

# from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

from numpy.random import seed
from tensorflow.random import set_seed

# Set the seed for reproducability
seed(42)
set_seed(42)

# Set the output path for saving the model
os.environ['CURRENT_PATH'] = os.getcwd()
output_path = os.path.join(os.environ['CURRENT_PATH'], 'output')

# visualization imports
import matplotlib.pyplot as plt
%matplotlib inline

# Load the datasets
posts_df = pd.read_csv('datasets/post_data.csv')
views_df = pd.read_csv('datasets/view_data.csv')
users_df = pd.read_csv('datasets/user_data.csv')

# 2. DATA PRE-PROCESSING
## 2.1. Generating Score Data

In [264]:
views_df_with_scores = pd.DataFrame(views_df)

# Generate random probabilities for 1, 2, and 3
probs = np.random.dirichlet(np.ones(3))

# Assign probabilities for the scores
scores = np.random.choice(
    [1, 2, 3],
    size=len(views_df),
    p=probs
)

# Add the score column to the dataframe
views_df_with_scores['score'] = scores

views_df_with_scores.head()

,user_id,post_id,time_stamp,score
0,5eece14ffc13ae660900008b,136781766,01/01/2019 01:30 PM,2
1,5eece14efc13ae660900003c,43094523,01/01/2019 01:33 PM,2
2,5eece14efc13ae6609000025,42428071,01/01/2019 01:43 PM,2
3,5eece14ffc13ae66090001d4,76472880,01/01/2019 01:54 PM,1
4,5eece14ffc13ae66090000ac,202721843,01/01/2019 02:00 PM,3


## 2.2. Data Cleaning and Transformation for Model Training

In [265]:
# Drop the columns that are not needed and remove the rows with missing title values
cleaned_data = views_df_with_scores.drop(['time_stamp'], axis=1)

# Drop the duplicates
cleaned_data = cleaned_data.drop_duplicates()

# Map user_id and post_id to numeric indices
cleaned_data["user_id"] = cleaned_data["user_id"].astype("category").cat.codes + 1
cleaned_data["post_id"] = cleaned_data["post_id"].astype("category").cat.codes + 1

cleaned_data.head()

,user_id,post_id,score
0,140,813,2
1,61,202,2
2,38,195,2
3,469,418,1
4,173,1217,3


In [266]:
unique_users_num = len(cleaned_data.user_id.unique())
unique_posts_num = len(cleaned_data.post_id.unique())

print('There are {} unique users and {} unique posts in this data set'.format(unique_users_num, unique_posts_num))

There are 501 unique users and 6001 unique posts in this data set


In [267]:
users_max_id = cleaned_data.user_id.max()
posts_max_id = cleaned_data.post_id.max()

print('There are {} distinct users, and the max of user_id is also {}'.format(unique_users_num, users_max_id))
print('There are {} distinct posts, and the max of post_id is also {}'.format(unique_posts_num, posts_max_id))

There are 501 distinct users, and the max of user_id is also 501
There are 6001 distinct posts, and the max of post_id is also 6001


## 2.3. Splitting Data into Train and Test Sets

In [268]:
df_train, df_test = train_test_split(cleaned_data, test_size=0.2, shuffle=True, random_state=99)
print('shape of training data set:')
print(df_train.shape)
print('shape of test data set:')
print(df_test.shape)

shape of training data set:
(56980, 3)
shape of test data set:
(14245, 3)


# 3. Model Training, Evaluation, and Weights Loading Functions

In [269]:
# Define the model training function
def train_model(model, optimizer, batch_size, num_epochs, validation_split, input_data, target_data, output_model_name):
    # Define a custom metric for Root Mean Squared Error (RMSE)
    def compute_rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_true - y_pred)))

    # Compile the model with the specified optimizer and loss function
    model.compile(
        optimizer=optimizer.lower(),
        loss='mean_squared_error',
        metrics=['mean_squared_error', compute_rmse]
    )

    # Define callbacks for early stopping and model checkpointing
    early_stopping = EarlyStopping(monitor='val_compute_rmse', patience=10, verbose=1)
    model_checkpoint = ModelCheckpoint(
        filepath=os.path.join(output_path, output_model_name),
        monitor='val_compute_rmse',
        save_best_only=True,
        save_weights_only=True
    )

    # Train the model and return the history
    history = model.fit(
        x=input_data,
        y=target_data,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=validation_split,
        callbacks=[early_stopping, model_checkpoint]
    )

    return history

# Define the model's weights loading function
def load_model_weights(model, weights_file_path):
    model.load_weights(weights_file_path)
    return model

# Define the model evaluation function
calculate_rmse = lambda true_values, predicted_values: np.sqrt(
    np.mean(np.square(np.squeeze(predicted_values) - np.squeeze(true_values)))
)

# 4. Training and Testing the Generalized Matrix Factorization Model
## 4.1. Defining the GMF Model Architecture

In [270]:
def build_gmf_model(num_users, num_posts, latent_dim, user_reg, post_reg):
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    post_input = Input(shape=(1,), dtype='int32', name='post_input')

    # Embedding layers
    user_embedding = Embedding(
        input_dim=num_users + 1,
        output_dim=latent_dim,
        embeddings_initializer='uniform',
        name='user_embedding',
        embeddings_regularizer=l2(user_reg),
        input_length=1
    )
    post_embedding = Embedding(
        input_dim=num_posts + 1,
        output_dim=latent_dim,
        embeddings_initializer='uniform',
        name='post_embedding',
        embeddings_regularizer=l2(post_reg),
        input_length=1
    )

    # Flatten embedding vectors
    user_latent = Flatten()(user_embedding(user_input))
    post_latent = Flatten()(post_embedding(post_input))

    # Combine user and post embeddings
    interaction_vector = Multiply()([user_latent, post_latent])

    # Output layer
    prediction = Dense(1, kernel_initializer='glorot_uniform', name='prediction')(interaction_vector)

    # Create and return the model
    model = Model([user_input, post_input], prediction)
    return model

In [271]:
latent_dim = 10
user_reg = 0
post_reg = 0

GMF_model = build_gmf_model(unique_users_num, unique_posts_num, latent_dim, user_reg, post_reg)
GMF_model.summary()

Model: "model_80"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 post_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 10)                5020      ['user_input[0][0]']          
                                                                                                  
 post_embedding (Embedding)  (None, 1, 10)                60020     ['post_input[0][0]']          
                                                                                           

## 4.2. Training the GMF Model

In [272]:
# model config
batch_size = 64
num_epochs = 100
validation_split = 0.25

# train model
history = train_model(GMF_model, 'adam', batch_size, num_epochs, validation_split, 
                      input_data=[df_train.user_id.values, df_train.post_id.values],
                      target_data=df_train.score.values,
                        output_model_name='best_gmf_model.hdf5')

Epoch 1/100
668/668 [==============================] - 3s 3ms/step - loss: 3.8202 - mean_squared_error: 3.8202 - compute_rmse: 1.9458 - val_loss: 2.7582 - val_mean_squared_error: 2.7582 - val_compute_rmse: 1.6592
Epoch 2/100
668/668 [==============================] - 2s 3ms/step - loss: 1.7230 - mean_squared_error: 1.7230 - compute_rmse: 1.2893 - val_loss: 0.7813 - val_mean_squared_error: 0.7813 - val_compute_rmse: 0.8818
Epoch 3/100
668/668 [==============================] - 2s 3ms/step - loss: 0.4412 - mean_squared_error: 0.4412 - compute_rmse: 0.6596 - val_loss: 0.4155 - val_mean_squared_error: 0.4155 - val_compute_rmse: 0.6426
Epoch 4/100
668/668 [==============================] - 2s 3ms/step - loss: 0.3112 - mean_squared_error: 0.3112 - compute_rmse: 0.5560 - val_loss: 0.4129 - val_mean_squared_error: 0.4129 - val_compute_rmse: 0.6405
Epoch 5/100
668/668 [==============================] - 2s 4ms/step - loss: 0.2603 - mean_squared_error: 0.2603 - compute_rmse: 0.5083 - val_loss: 0.

## 4.3. Loading the Trained GMF Model and Evaluating Performance

In [273]:
# Load the pre-trained GMF model with the best weights
gmf_model = build_gmf_model(unique_users_num, unique_posts_num, latent_dim, user_reg, post_reg)
gmf_model = load_model_weights(gmf_model, os.path.join(output_path, 'best_gmf_model.hdf5'))

# Generate predictions using the test data
predicted_scores = gmf_model.predict([df_test.user_id.values, df_test.post_id.values])

# Calculate the RMSE for the predictions
rmse_error = calculate_rmse(df_test.score.values, predicted_scores)

# Print the RMSE result
print('The out-of-sample RMSE of rating predictions is', round(rmse_error, 4))

446/446 [==============================] - 1s 1ms/step
The out-of-sample RMSE of rating predictions is 0.641


# 5. Training and Testing the Multi-Layer Perceptron Model
## 5.1. Defining the MLP Model Architecture

In [274]:
def build_mlp_model(num_users, num_posts, layers, reg_layers):
    # Ensure the number of layers matches the number of regularization parameters
    assert len(layers) == len(reg_layers)
    num_layer = len(layers)  # Number of layers in the MLP

    # Define input layers for user and post IDs
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    post_input = Input(shape=(1,), dtype='int32', name='post_input')

    # Define embedding layers for users and posts
    user_embedding = Embedding(
        input_dim=num_users + 1,           
        output_dim=layers[0] // 2,        
        embeddings_initializer='uniform', 
        name='user_embedding',           
        embeddings_regularizer=l2(reg_layers[0]),  
        input_length=1)                   
    
    post_embedding = Embedding(
        input_dim=num_posts + 1,         
        output_dim=layers[0] // 2,       
        embeddings_initializer='uniform', 
        name='post_embedding',           
        embeddings_regularizer=l2(reg_layers[0]), 
        input_length=1)                   

    # Flatten the embeddings to prepare for concatenation
    user_latent = Flatten()(user_embedding(user_input))  
    post_latent = Flatten()(post_embedding(post_input))  

    # Concatenate the user and item embedding vectors
    vector = Concatenate(axis=-1)([user_latent, post_latent])

    # Add fully connected (dense) layers
    for idx in range(1, num_layer): 
        layer = Dense(
            units=layers[idx],                   
            activation='relu',                   
            kernel_initializer='glorot_uniform',
            kernel_regularizer=l2(reg_layers[idx]), 
            name=f'layer{idx}')                 
        vector = layer(vector) 
    
    # Add the final prediction layer with a single output
    prediction = Dense(1, kernel_initializer='glorot_uniform', name='prediction')(vector)

    # Create the model with user and post inputs and prediction as the output
    model = Model([user_input, post_input], prediction)

    return model

In [275]:
layers = [64, 32, 16, 8]
reg_layers = [0, 0, 0, 0]

MLP_model = build_mlp_model(unique_users_num, unique_posts_num, layers, reg_layers)
MLP_model.summary()

Model: "model_82"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 post_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 32)                16064     ['user_input[0][0]']          
                                                                                                  
 post_embedding (Embedding)  (None, 1, 32)                192064    ['post_input[0][0]']          
                                                                                           

## 5.2. Training the MLP Model

In [276]:
# model config
batch_size = 64
num_epochs = 100
validation_split = 0.25

# train model
history = train_model(MLP_model, 'adam', batch_size, num_epochs, validation_split, 
                      input_data=[df_train.user_id.values, df_train.post_id.values],
                      target_data=df_train.score.values,
                        output_model_name='best_mlp_model.hdf5')

Epoch 1/100
668/668 [==============================] - 4s 5ms/step - loss: 0.6943 - mean_squared_error: 0.6943 - compute_rmse: 0.7315 - val_loss: 0.3512 - val_mean_squared_error: 0.3512 - val_compute_rmse: 0.5908
Epoch 2/100
668/668 [==============================] - 3s 4ms/step - loss: 0.3370 - mean_squared_error: 0.3370 - compute_rmse: 0.5785 - val_loss: 0.3506 - val_mean_squared_error: 0.3506 - val_compute_rmse: 0.5902
Epoch 3/100
668/668 [==============================] - 3s 4ms/step - loss: 0.3202 - mean_squared_error: 0.3202 - compute_rmse: 0.5639 - val_loss: 0.3578 - val_mean_squared_error: 0.3578 - val_compute_rmse: 0.5962
Epoch 4/100
668/668 [==============================] - 3s 4ms/step - loss: 0.3011 - mean_squared_error: 0.3011 - compute_rmse: 0.5469 - val_loss: 0.3742 - val_mean_squared_error: 0.3742 - val_compute_rmse: 0.6097
Epoch 5/100
668/668 [==============================] - 3s 4ms/step - loss: 0.2735 - mean_squared_error: 0.2735 - compute_rmse: 0.5208 - val_loss: 0.

## 5.3. Loading the Trained MLP Model and Evaluating Performance

In [277]:
# Load the pre-trained MLP model with the best weights
mlp_model = build_mlp_model(unique_users_num, unique_posts_num, layers, reg_layers)
mlp_model = load_model_weights(mlp_model, os.path.join(output_path, 'best_mlp_model.hdf5'))

# Generate predictions using the test data
predicted_scores = mlp_model.predict([df_test.user_id.values, df_test.post_id.values])

# Calculate the RMSE for the predictions
rmse_error = calculate_rmse(df_test.score.values, predicted_scores)

# Print the RMSE result
print('The out-of-sample RMSE of rating predictions is', round(rmse_error, 4))

446/446 [==============================] - 1s 1ms/step
The out-of-sample RMSE of rating predictions is 0.589


# 6. Training and Testing the Neural Matrix Factorization Model
## 6.1. Defining the NeuMF Model Architecture

In [278]:
def build_neumf_model(num_users, num_posts, MF_dim, MF_reg, MLP_layers, MLP_regs):
    # Ensure the number of layers matches the number of regularization parameters
    assert len(MLP_layers) == len(MLP_regs)
    num_MLP_layer = len(MLP_layers) # Number of layers in the MLP

    # Define input layers for user and post IDs
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    post_input = Input(shape=(1,), dtype='int32', name='post_input')

    # Embedding layers for MF
    mf_user_embedding = Embedding(
        input_dim=num_users + 1,
        output_dim=MF_dim,
        embeddings_initializer='uniform',
        name='mf_user_embedding',
        embeddings_regularizer=l2(MF_reg[0]),
        input_length=1)
    mf_post_embedding = Embedding(
        input_dim=num_posts + 1,
        output_dim=MF_dim,
        embeddings_initializer='uniform',
        name='mf_post_embedding',
        embeddings_regularizer=l2(MF_reg[1]),
        input_length=1)
    
    # Embedding layers for MLP
    mlp_user_embedding = Embedding(
        input_dim=num_users + 1,
        output_dim=MLP_layers[0] // 2,
        embeddings_initializer='uniform',
        name='mlp_user_embedding',
        embeddings_regularizer=l2(MLP_regs[0]),
        input_length=1)
    mlp_post_embedding = Embedding(
        input_dim=num_posts + 1,
        output_dim=MLP_layers[0] // 2,
        embeddings_initializer='uniform',
        name='mlp_post_embedding',
        embeddings_regularizer=l2(MLP_regs[0]),
        input_length=1) 
    
    # Flatten the embeddings to prepare for concatenation
    mf_user_latent = Flatten()(mf_user_embedding(user_input))
    mf_post_latent = Flatten()(mf_post_embedding(post_input))
    mf_vector = Multiply()([mf_user_latent, mf_post_latent])

    # Flatten the embeddings to prepare for concatenation
    mlp_user_latent = Flatten()(mlp_user_embedding(user_input))
    mlp_post_latent = Flatten()(mlp_post_embedding(post_input))
    mlp_vector = Concatenate(axis=-1)([mlp_user_latent, mlp_post_latent])
    
    # Concatenate the two latent vectors
    # Add fully connected (dense) layers
    for idx in range(1, num_MLP_layer):
        layer = Dense(
            units=MLP_layers[idx],
            activation='relu',
            kernel_initializer='glorot_uniform',
            kernel_regularizer=l2(MLP_regs[idx]),
            name = 'layer%d' %idx)
        mlp_vector = layer(mlp_vector)
    
    predict_vector = Concatenate(axis=-1)([mf_vector, mlp_vector])

    # Concatenate the two latent vectors
    prediction = Dense(1, kernel_initializer='glorot_uniform', name='prediction')(predict_vector)
    
    # Create the model with user and post inputs and prediction as the output
    model = Model([user_input, post_input], prediction)
    
    return model

In [279]:
MF_dim = 10
MF_reg = (0, 0)
MLP_layers = [64, 32, 16, 8]
MLP_regs = [0, 0, 0, 0]

NeuMF_model = build_neumf_model(
    num_users=unique_users_num,
    num_posts=unique_posts_num,
    MF_dim=MF_dim,
    MF_reg=MF_reg,
    MLP_layers=MLP_layers,
    MLP_regs=MLP_regs
)
NeuMF_model.summary()

Model: "model_84"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 post_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 mlp_user_embedding (Embedd  (None, 1, 32)                16064     ['user_input[0][0]']          
 ing)                                                                                             
                                                                                                  
 mlp_post_embedding (Embedd  (None, 1, 32)                192064    ['post_input[0][0]']   

## 6.2. Training the NeuMF Model

In [280]:
# model config
batch_size = 64
num_epochs = 100
validation_split = 0.25

# train model
history = train_model(NeuMF_model, 'adam', batch_size, num_epochs, validation_split, 
                        input_data=[df_train.user_id.values, df_train.post_id.values],
                        target_data=df_train.score.values,
                        output_model_name='best_neumf_model.hdf5')

Epoch 1/100
668/668 [==============================] - 5s 5ms/step - loss: 0.6565 - mean_squared_error: 0.6565 - compute_rmse: 0.7170 - val_loss: 0.3486 - val_mean_squared_error: 0.3486 - val_compute_rmse: 0.5886
Epoch 2/100
668/668 [==============================] - 3s 5ms/step - loss: 0.3316 - mean_squared_error: 0.3316 - compute_rmse: 0.5739 - val_loss: 0.3520 - val_mean_squared_error: 0.3520 - val_compute_rmse: 0.5914
Epoch 3/100
668/668 [==============================] - 3s 5ms/step - loss: 0.2698 - mean_squared_error: 0.2698 - compute_rmse: 0.5175 - val_loss: 0.3730 - val_mean_squared_error: 0.3730 - val_compute_rmse: 0.6089
Epoch 4/100
668/668 [==============================] - 3s 5ms/step - loss: 0.1678 - mean_squared_error: 0.1678 - compute_rmse: 0.4078 - val_loss: 0.4261 - val_mean_squared_error: 0.4261 - val_compute_rmse: 0.6507
Epoch 5/100
668/668 [==============================] - 3s 5ms/step - loss: 0.1038 - mean_squared_error: 0.1038 - compute_rmse: 0.3202 - val_loss: 0.

## 6.3. Loading the Trained NeuMF Model and Evaluating Performance

In [281]:
# Load the pre-trained NeuMF model with the best weights
neumf_model = build_neumf_model(
    num_users=users_max_id,
    num_posts=posts_max_id,
    MF_dim=MF_dim,
    MF_reg=MF_reg,
    MLP_layers=MLP_layers,
    MLP_regs=MLP_regs
)
neumf_model = load_model_weights(neumf_model, os.path.join(output_path, 'best_neumf_model.hdf5'))

# Generate predictions using the test data
predicted_scores = neumf_model.predict([df_test.user_id.values, df_test.post_id.values])

# Calculate the RMSE for the predictions
rmse_error = calculate_rmse(df_test.score.values, predicted_scores)

# Print the RMSE result
print('The out-of-sample RMSE of rating predictions is', round(rmse_error, 4))

446/446 [==============================] - 1s 1ms/step
The out-of-sample RMSE of rating predictions is 0.5868
